## Deliverable 2. Create a Customer Travel Destinations Map.

In [23]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import sys

# Import API key
path_config = '../config/'
sys.path.append(path_config)
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [24]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current Description
0,0,Busselton,-33.6500,115.3333,61.00,99,99,1.01,AU,2021-04-11 22:31:19,overcast clouds
1,1,Atuona,-9.8000,-139.0333,80.60,77,18,17.02,PF,2021-04-11 22:31:19,few clouds
2,2,Barbacoas,1.6715,-78.1398,77.72,93,99,2.33,CO,2021-04-11 22:31:19,light rain
3,3,Rikitea,-23.1203,-134.9692,78.30,74,7,8.19,PF,2021-04-11 22:28:26,light rain
4,4,Vaini,-21.2000,-175.2000,80.60,69,20,5.75,TO,2021-04-11 22:31:20,few clouds


In [25]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 95


In [26]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
#filter the dataset to find the cities that fit the criteria
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                      (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current Description
0,0,Busselton,-33.6500,115.3333,61.00,99,99,1.01,AU,2021-04-11 22:31:19,overcast clouds
1,1,Atuona,-9.8000,-139.0333,80.60,77,18,17.02,PF,2021-04-11 22:31:19,few clouds
2,2,Barbacoas,1.6715,-78.1398,77.72,93,99,2.33,CO,2021-04-11 22:31:19,light rain
3,3,Rikitea,-23.1203,-134.9692,78.30,74,7,8.19,PF,2021-04-11 22:28:26,light rain
4,4,Vaini,-21.2000,-175.2000,80.60,69,20,5.75,TO,2021-04-11 22:31:20,few clouds


In [27]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                393
City                   393
Lat                    393
Lng                    393
Max Temp               393
Humidity               393
Cloudiness             393
Wind Speed             393
Country                387
Date                   393
Current Description    393
dtype: int64

In [28]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                387
City                   387
Lat                    387
Lng                    387
Max Temp               387
Humidity               387
Cloudiness             387
Wind Speed             387
Country                387
Date                   387
Current Description    387
dtype: int64

In [29]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Busselton,AU,61.00,overcast clouds,-33.6500,115.3333,
1,Atuona,PF,80.60,few clouds,-9.8000,-139.0333,
2,Barbacoas,CO,77.72,light rain,1.6715,-78.1398,
3,Rikitea,PF,78.30,light rain,-23.1203,-134.9692,
4,Vaini,TO,80.60,few clouds,-21.2000,-175.2000,
8,Bambous Virieux,MU,78.01,broken clouds,-20.3428,57.7575,
11,Saldanha,ZA,60.80,overcast clouds,-33.0117,17.9442,
12,Mandali,IQ,64.33,clear sky,33.7499,45.5541,
13,Hobart,AU,50.00,broken clouds,-42.8794,147.3294,
14,Baherden,TM,65.80,broken clouds,38.4362,57.4316,


In [30]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found ... skipping.")
        

Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.


In [44]:
hotel_df["Hotel Name"].value_counts()

                                        25
Hospitality Carnarvon                    1
Holiday Inn Express & Suites Terrell     1
Estación de Chagres                      1
Rancho El Sauce y la Palma               1
                                        ..
Veranda Paul et Virginie Hotel & Spa     1
Quality Inn                              1
Yingbin Restaurant                       1
ibis Styles Karratha                     1
Sura Dare Village                        1
Name: Hotel Name, Length: 363, dtype: int64

In [54]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df['Hotel Name'] != ""]
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Busselton,AU,61.00,overcast clouds,-33.6500,115.3333,Observatory Guest House
1,Atuona,PF,80.60,few clouds,-9.8000,-139.0333,Villa Enata
2,Barbacoas,CO,77.72,light rain,1.6715,-78.1398,edificio justin
3,Rikitea,PF,78.30,light rain,-23.1203,-134.9692,Pension Maro'i
4,Vaini,TO,80.60,few clouds,-21.2000,-175.2000,Keleti Beach Resort


In [55]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [58]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [59]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))